In [1]:
import pandas as pd
import numpy as np

In [2]:
heart = pd.read_csv('/home/tiago/Downloads/insurance.csv')
heart.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
heart.shape

(1338, 7)

In [4]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
heart.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [6]:
heart.value_counts()

age  sex     bmi     children  smoker  region     charges    
19   male    30.590  0         no      northwest  1639.56310     2
47   male    29.830  3         no      northwest  9620.33070     1
48   female  25.850  3         yes     southeast  24180.93350    1
             22.800  0         no      southwest  8269.04400     1
47   male    47.520  1         no      southeast  8083.91980     1
                                                                ..
31   female  25.740  0         no      southeast  3756.62160     1
             23.600  2         no      southwest  4931.64700     1
             21.755  0         no      northwest  4134.08245     1
30   male    44.220  2         no      southeast  4266.16580     1
64   male    40.480  0         no      southeast  13831.11520    1
Length: 1337, dtype: int64

In [7]:
# Import label encoder
from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
heart['sex']= label_encoder.fit_transform(heart['sex'])
heart['smoker']= label_encoder.fit_transform(heart['smoker'])
heart['region']= label_encoder.fit_transform(heart['region'])


In [8]:
X, y = heart.loc[:, heart.columns != 'charges'], heart['charges']

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=30,random_state=42)

In [10]:
print("x_train shape is:",X_train.shape)

x_train shape is: (1308, 6)


In [11]:
print("y_train shape is:",y_train.shape)

y_train shape is: (1308,)


In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
#XGBoost with Bagging and Boosting
import xgboost as xgb
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error

# Init classifier
xgb_cl = xgb.XGBRegressor()

# The baggging ensemble classifier is initialized with:

bagging = BaggingRegressor(base_estimator=xgb_cl, n_estimators=5, max_samples=50, bootstrap=True)

# Training
bagging.fit(X_train, y_train)

# Evaluating
print(f"Train score: {bagging.score(X_train, y_train)}")
print(f"Test score: {bagging.score(X_test, y_test)}")

# Fit
xgb_cl.fit(X_train, y_train)

# Predict
preds = xgb_cl.predict(X_test)

print("Mean Squared Error Value:",mean_squared_error(y_test, preds, multioutput='uniform_average'))

Train score: 0.8172501879455122
Test score: 0.9389474712447533
Mean Squared Error Value: 9688985.644590309


In [16]:
#Decision Tree with Bagging and Boosting
from sklearn.tree import DecisionTreeRegressor
# Import Decision Tree Classifier
from sklearn.ensemble import BaggingRegressor
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics 
from sklearn.metrics import mean_squared_error

tree = DecisionTreeRegressor(max_depth=3, random_state=23)

# The baggging ensemble classifier is initialized with:

bagging = BaggingRegressor(base_estimator=tree, n_estimators=5, max_samples=50, bootstrap=True)

# Training
bagging.fit(X_train, y_train)

# Evaluating
print(f"Train score: {bagging.score(X_train, y_train)}")
print(f"Test score: {bagging.score(X_test, y_test)}")

print('Random Forest Regressor Train Score is : ' , bagging.score(X_train, y_train))
print('Random Forest Regressor Test Score is : ' , bagging.score(X_test, y_test))
print('Random Forest Regressor No. of features are : ' , bagging.n_features_)
print('----------------------------------------------------')

y_pred = bagging.predict(X_test)
print('Predicted Value for Random Forest Regressor is : ' , y_pred)

# Train Decision Tree Classifer
tree = tree.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = tree.predict(X_test)

print("Mean Squared Error Value:",mean_squared_error(y_test, preds, multioutput='uniform_average'))

Train score: 0.82794680734351
Test score: 0.9110661289525561
Random Forest Regressor Train Score is :  0.82794680734351
Random Forest Regressor Test Score is :  0.9110661289525561
Random Forest Regressor No. of features are :  6
----------------------------------------------------
Predicted Value for Random Forest Regressor is :  [12088.17398633  6094.19642028 22388.37451633 10609.62329989
 37982.10233967 10843.22380924  4443.52328513 17953.62419114
  6094.19642028  8465.10390178 22144.39972333  6094.19642028
  6094.19642028 45831.69071167 47686.00044733 45831.69071167
 14692.68197719 45831.69071167 12088.17398633 21697.29363133
  8161.50446763  9505.81331428  4443.52328513  9192.55067409
  9943.65458822 13599.87832699 19432.17487759  5521.81539737
 12088.17398633  4443.52328513]
Mean Squared Error Value: 9688985.644590309


/home/tiago/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


In [19]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print ("Mean Squared Error Value : ", mean_squared_error(y_test, y_pred, multioutput='uniform_average'))

Mean Squared Error Value :  27132085.232885268


In [39]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(12, input_dim=6, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))
# compile the keras model
model.compile(loss='mse', optimizer='adam')
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=150, batch_size=10)

Epoch 1/150
131/131 [==============================] - 1s 2ms/step - loss: 319915168.0000
Epoch 2/150
131/131 [==============================] - 0s 2ms/step - loss: 248145152.0000
Epoch 3/150
131/131 [==============================] - 0s 2ms/step - loss: 55548968.0000
Epoch 4/150
131/131 [==============================] - 0s 2ms/step - loss: 36848928.0000
Epoch 5/150
131/131 [==============================] - 0s 2ms/step - loss: 33959688.0000
Epoch 6/150
131/131 [==============================] - 0s 2ms/step - loss: 32372438.0000
Epoch 7/150
131/131 [==============================] - 0s 2ms/step - loss: 31602140.0000
Epoch 8/150
131/131 [==============================] - 0s 2ms/step - loss: 30857414.0000
Epoch 9/150
131/131 [==============================] - 0s 2ms/step - loss: 30106562.0000
Epoch 10/150
131/131 [==============================] - 0s 2ms/step - loss: 29522660.0000
Epoch 11/150
131/131 [==============================] - 0s 3ms/step - loss: 29102650.0000
Epoch 12/150
131/

131/131 [==============================] - 0s 3ms/step - loss: 22812866.0000
Epoch 93/150
131/131 [==============================] - 0s 2ms/step - loss: 22852284.0000
Epoch 94/150
131/131 [==============================] - 0s 2ms/step - loss: 22897472.0000
Epoch 95/150
131/131 [==============================] - 0s 2ms/step - loss: 22907938.0000
Epoch 96/150
131/131 [==============================] - 0s 2ms/step - loss: 22709850.0000
Epoch 97/150
131/131 [==============================] - 0s 2ms/step - loss: 22871526.0000
Epoch 98/150
131/131 [==============================] - 0s 2ms/step - loss: 22598912.0000
Epoch 99/150
131/131 [==============================] - 0s 2ms/step - loss: 22950518.0000
Epoch 100/150
131/131 [==============================] - 0s 2ms/step - loss: 22724966.0000
Epoch 101/150
131/131 [==============================] - 0s 2ms/step - loss: 22710246.0000
Epoch 102/150
131/131 [==============================] - 0s 2ms/step - loss: 22673184.0000
Epoch 103/150
131/13

In [30]:
#metrics XGBoosting
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
print("R^2 : ", r2_score(y_test, preds))
print("MAE :", mean_absolute_error(y_test,preds))
print("RMSE:",np.sqrt(mean_squared_error(y_test, preds)))

R^2 :  0.9512410400184247
MAE : 1892.62865024292
RMSE: 3112.7135500380227


In [41]:
#metrics NN
predictions = model.predict(X_test)
print("R^2 : ", r2_score(y_test, predictions))
print("MAE :", mean_absolute_error(y_test,predictions))
print("RMSE:",np.sqrt(mean_squared_error(y_test, predictions)))

R^2 :  0.9702163407826124
MAE : 1543.1740101302084
RMSE: 2432.769522892851
